## Simulating Frames on top of Omega ##

## Scripting the Omega-Frame Test

In [1]:
import os
from pathlib import Path

testfolder = r'C:\Users\sarefeen\Documents\RadianceScenes\Omega'  

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sarefeen\Documents\RadianceScenes\Omega


In [2]:
import bifacial_radiance

In [3]:
demo = bifacial_radiance.RadianceObj('bifacial_example_omegatest',str(testfolder)) 

path = C:\Users\sarefeen\Documents\RadianceScenes\Omega


In [4]:
albedo = 0.62
demo.setGround(albedo)

Loading albedo, 1 value(s), 0.620 avg
1 nonzero albedo values.


In [5]:
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)

Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


In [6]:
metdata = demo.readWeatherFile(epwfile, coerce_year = 2021)

start: 2021-01-01 01:00:00
end: 2021-12-31 23:00:00
Saving file EPWs\epw_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [ ]:
#generating sky
demo.gendaylit(4020)

In [ ]:
# mod_adj_piece

x,y,z = 1,2,0 #module parameters

x_omega1 = 0.4 #assuming 10 cm
mod_overlap = 0.2 # assuming 3 cm overlap
y_omega = 1.5
z_omega1 = 0.05
#x_start_m = -x/2 + mod_overlap 
x_translate1 = -x/2 - x_omega1 + mod_overlap
y_translate = -y_omega/2 #common for all the pieces
z_translate1 = -z_omega1
x_size_1 = x_omega1
y_size_1 = y_omega
z_size_1 = z_omega1 #assumed for nominal thickness

In [ ]:
# vertical piece
x_omega2 = 0.05
z_omega2 = 0.5 #assuming 50 cm
# z_mod_thickness = 0.02 #2 cm thickness for module
x_size_2 = x_omega2
# z_start = -z_mod_thickness
x_translate2 = x_translate1
z_translate2 = -z_omega2
z_size_2 = z_omega2
y_size_2 = y_omega


In [ ]:
# TT_adj_piece

x_omega3 = 0.25
z_omega3 = 0.05
#x_start_tt = -x/2 - x_omega3 -x_omega1 + mod_overlap
#x_end_tt = -x/2 - x_omega/2 + mod_overlap - x_omega/2
#x_end_tt = -x/2 - x_omega1 - x_omega3
x_size_3 = x_omega3
y_size_3 = y_omega
z_size_3 = z_omega3 #assumed for nominal thickness

x_translate3 = x_translate1-x_omega3
z_translate3 = z_translate2


In [ ]:
#making the geometry

x,y,z = 1,2,0
name1 = 'mod_adj'
name2 = 'verti'
name3 = 'tt_adj'
offsetfromaxis = True
omega_material = 'litesoil'
frame = True
nameframe1 = 'Side Frame'
nameframe2 = 'Frame Leg'
frame_material = 'Metal_Grey'
frame_side_x = 0.05
frame_side_z = 0.3
frame_y = y
frame_leg_x = mod_overlap*0.8
frame_leg_z = 0.1

# making frames: east side

custom_text = '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(frame_material, nameframe1, frame_side_x, frame_y, frame_side_z, -x/2-frame_side_x, y_translate, -frame_side_z) 
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(frame_material, nameframe2, frame_leg_x, frame_y, frame_leg_z, -x/2-frame_side_x, y_translate, -frame_side_z)


# making frames: west side

custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(frame_material, nameframe1, frame_side_x, frame_y, frame_side_z, x/2, y_translate, -frame_side_z) 
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(frame_material, nameframe2, frame_leg_x, frame_y, frame_leg_z, -x/2-frame_side_x, y_translate, -frame_side_z)


#customizing the West side of the module for omega

custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name1, x_size_1, y_size_1, z_size_1, x_translate1, y_translate, z_translate1-frame_side_z) 
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name2, x_size_2, y_size_2, z_size_2, x_translate2, y_translate, z_translate2-frame_side_z)
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name3, x_size_3, y_size_3, z_size_3, x_translate3, y_translate, z_translate3-frame_side_z)

#customizing the East side of the module for omega

custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name1, x_size_1, y_size_1, z_size_1, -x_translate1-x_omega1, y_translate, z_translate1-frame_side_z) 
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name2, x_size_2, y_size_2, z_size_2, -x_translate2-x_omega2, y_translate, z_translate2-frame_side_z)
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name3, x_size_3, y_size_3, z_size_3, -x_translate3-x_omega3, y_translate, z_translate3-frame_side_z)


#custom_text = '! genbox black CenterMarker 0.1 0.1 4 | xform -t -0.05 -0.05 0'
#text +='| xform -t {} {} {} '.format(-x/2.0, (-yNy/2.0)-(ygap(Ny-1)/2.0), offsetfromaxis) 
#text += '-a {} -t 0 {} 0'.format(Ny, y+ygap) 

# have to find out how pipe and -a changes the structure


In [ ]:
#with frames

frame_z= 0.3

# the module will be frame_z elevated from the top surface of the omega
# 


In [ ]:
# creating the inverted omega scenario

x,y,z = 1,2,0
name1 = 'mod_adj'
name2 = 'verti'
name3 = 'tt_adj'
offsetfromaxis = True
omega_material = 'litesoil'

# shifting the non-inv omega shape of west as inv omega shape of east
x_translate1_inv_east = x/2-mod_overlap
x_shift_east = x_translate1_inv_east - x_translate1

# shifting the non-inv omega shape of west as inv omega shape of east
x_translate1_inv_west = -x_translate1_inv_east - x_omega1
x_shift_west = -x_translate1_inv_west + (-x_translate1-x_omega1)

In [ ]:
#customizing the East side of the module for omega_inverted

custom_text = '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name1, x_size_1, y_size_1, z_size_1, x_translate1_inv_east, y_translate, z_translate1) 
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name2, x_size_2, y_size_2, z_size_2, x_translate2 + x_shift_east, y_translate, z_translate2)
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name3, x_size_3, y_size_3, z_size_3, x_translate3 + x_shift_east, y_translate, z_translate3)

#customizing the West side of the module for omega_inverted

custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name1, x_size_1, y_size_1, z_size_1, x_translate1_inv_west, y_translate, z_translate1) 
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name2, x_size_2, y_size_2, z_size_2, -x_translate2-x_omega2 -x_shift_west, y_translate, z_translate2)
custom_text += '\r\n! genbox {} {} {} {} {} | xform -t {} {} {}'.format(omega_material, name3, x_size_3, y_size_3, z_size_3, -x_translate3-x_omega3 - x_shift_west, y_translate, z_translate3)


In [ ]:
xgap = 0.05
ygap = 0.05
zgap = 0.5

module_type = 'Prism Solar Bi60 landscape' 
demo.makeModule(name=module_type,x=x, y=y, torquetube = True, diameter = 0.3, xgap = xgap, ygap = ygap, zgap = zgap, customtext = custom_text, axisofrotationTorqueTube=False)

In [ ]:
nMods = 1
nRows = 1
sceneDict = {'tilt':0, 'pitch':3, 'clearance_height':3,'azimuth':90, 'nMods': nMods, 'nRows': nRows} 

In [ ]:
scene = demo.makeScene(module_type,sceneDict)

In [ ]:
octfile = demo.makeOct()

In [ ]:
#rvu command from tutorial 1
rvu -vf views\front.vp -e .01 bifacial_example_omegatest.oct

rvu -vp 2.5 0 3 -vd -1 0 0 bifacial_example_omegatest.oct

rvu -vp 2.5 0 3 -vd 0 1 0 bifacial_example_omegatest.oct
